In [1]:
!pip install -q openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [3]:
#import openai
import os
from openai import OpenAI
from google.colab import userdata


key = userdata.get('OLSAI_API')

client = OpenAI(
    # This is the default and can be omitted
    api_key = key
)


In [4]:
# Text completion
def my_ai_assistant(prompt):
    response = client.chat.completions.create(
       messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4-turbo",
)
    return response.choices[0].message

# Example usage
prompt = "Translate the following English text to French: 'Hello, how are you?'"
result = my_ai_assistant(prompt)
print("My AI's response:", result)

My AI's response: ChatCompletionMessage(content='Bonjour, comment vas-tu ?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [5]:
# Function to upload a dataset
from pathlib import Path

file = client.files.create(
    file =  Path('sample_data/mnist_train_small.csv'),
    purpose='assistants'
)
file.id

'file-FL0ku6p5v0JRUJtsjPakeXbo'

In [6]:
# Create an assistant for the file ID
assistant = client.beta.assistants.create(
  name="OLSAI",
  model="gpt-4o",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)
print(assistant)

Assistant(id='asst_Pq0uRjh7iDWUIJapftVupUZ5', created_at=1723538817, description=None, instructions=None, metadata={}, model='gpt-4o', name='OLSAI', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-FL0ku6p5v0JRUJtsjPakeXbo']), file_search=None), top_p=1.0)


In [8]:
# Create an empty thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_Mnd0r2NUz5viDoQGBGanAEss', created_at=1723538843, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [13]:
thread_message = client.beta.threads.messages.create(
  thread.id,
  role="user",
  content="Make summary statistics for the dataset",
  attachments=[
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }]
)
print(thread_message)

Message(id='msg_P1uSBxnIWzmuCrBGAl5gxM1O', assistant_id=None, attachments=[Attachment(file_id='file-FL0ku6p5v0JRUJtsjPakeXbo', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Make summary statistics for the dataset'), type='text')], created_at=1723538994, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Mnd0r2NUz5viDoQGBGanAEss')


In [14]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Student."
)

In [15]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_WGCgv18ff66Z1DwJihvpnvHG', assistant_id='asst_Pq0uRjh7iDWUIJapftVupUZ5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='We have already generated the summary statistics for the dataset in a previous step. Here they are again for your reference:\n\n#### Summary Statistics (First Few Columns)\n\n|           | 6          | 0        | 0.1      | 0.2      | 0.3      | 0.4      | 0.5      | … |\n|-----------|------------|----------|----------|----------|----------|----------|----------|---|\n| **count** | 19999.000000 | 19999.0  | 19999.0  | 19999.0  | 19999.0  | 19999.0  | 19999.0  | ... |\n| **mean**  | 4.470124   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **std**   | 2.892807   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **min**   | 0.000000   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **25%**   | 

In [16]:
messages.data[0].content

[TextContentBlock(text=Text(annotations=[], value='We have already generated the summary statistics for the dataset in a previous step. Here they are again for your reference:\n\n#### Summary Statistics (First Few Columns)\n\n|           | 6          | 0        | 0.1      | 0.2      | 0.3      | 0.4      | 0.5      | … |\n|-----------|------------|----------|----------|----------|----------|----------|----------|---|\n| **count** | 19999.000000 | 19999.0  | 19999.0  | 19999.0  | 19999.0  | 19999.0  | 19999.0  | ... |\n| **mean**  | 4.470124   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **std**   | 2.892807   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **min**   | 0.000000   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **25%**   | 2.000000   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | 0.0      | ... |\n| **50%**   | 4.000000   | 0.0      | 0.0      | 0.0      | 0.0      | 0.0  

In [17]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

for step in run_steps.data:

  # If step involves code execution, print the code
  if step.type == 'code_interpreter':
      print(f"Python Code Executed: {step.step_details['code_interpreter']['input']}")
